In [1]:
import requests
import sqlite3
import datetime
from bs4 import BeautifulSoup

con = sqlite3.connect("cafe.db")
cur = con.cursor()
cur.execute('select * from data where type is "1"')
schizophrenia_free = cur.fetchall()
cur.execute('select * from data where type is "2"')
schizophrenia_qna = cur.fetchall()
cur.execute('select * from data where type is "3"')
adhd_free = cur.fetchall()
cur.execute('select * from data where type is "4"')
adhd_anonymous = cur.fetchall()
con.commit()
con.close()

con = sqlite3.connect("cafe_data.db")
cur = con.cursor()
cur.execute('select * from data where category is "1"')
sad = cur.fetchall()
con.commit()
con.close()

In [7]:
con = sqlite3.connect("raw_data.db")
cur = con.cursor()

cur.execute('select * from data where category is "5" and Title not like "%정서장애%" and Content not like "%정서장애%" and Content not like "%감정장애%" and Content not like "%기분장애%" and Content not like "%가을타다%" and Content not like "%청동장애%" and Content not like "%계절성%" and \
Title not like "%ADHD%" and Content not like "%ADHD%" and Content not like "%주의력%" and Content not like "%부주의%" and Content not like "%주의점%" and Content not like "%산만%" and Content not like "%과잉행동%" and Content not like "%충동%"  and Content not like "%수다%" and \
Title not like "%조현병%" and Content not like "%조현병%" and Content not like "%망상%" and Content not like "%환각%" and Content not like "%사고장애%" and Content not like "%정신운동장애%" and Content not like "%인사불성%" and Content not like "%강직%" and Content not like "%따라함%" and \
Title not like "%강박장애%" and Content not like "%강박장애%" and Content not like "%강박%" and Content not like "%제자리%" and Content not like "%집착%" and Content not like "%결벽증%" and Content not like "%결벽%" and Content not like "%깔끔%" and Content not like "%원위치%" and \
Title not like "%인격장애%" and Content not like "%인격장애%" and Content not like "%성격장애%" and Content not like "%변덕%" and Content not like "%불안정%" and Content not like "%양가감정%" and Content not like "%자기애%" and Content not like "%자기합리%" and Content not like "%분노조절%" and \
Title not like "%외상후%" and Content not like "%외상후%" and Content not like "%과민%" and Content not like "%천재지변%" and Content not like "%화재%" and Content not like "%폭행%" and Content not like "%강간%" and Content not like "%성추행%" and Content not like "%스트레스장애%" and Content not like "%PTSD%" and \
Title not like "%조울증%" and Content not like "%조울증%" and Content not like "%양극성%" and Content not like "%과대망상%" and Content not like "%조증%" and Content not like "%경조증%" and Content not like "%혼합삽화%" and \
Title not like "%섭식장애%" and Content not like "%섭식장애%" and Content not like "%폭식%" and Content not like "%거식%" and Content not like "%식이장애%" and Content not like "%식욕부진%" and Content not like "%대식증%" and Content not like "%과식%" and \
Title not like "%불안장애%" and Content not like "%불안장애%" and Content not like "%불안%" and Content not like "%공포%" and Content not like "%불면증%" and Content not like "%수면장애%" and \
Title not like "%공황장애%" and Content not like "%공황장애%" and \
Title not like "%우울증%" and Content not like "%우울증%" and Content not like "%우울%" and Content not like "%슬픔%" and Content not like "%무기력%" and Content not like "%자살%" and Content not like "%죄책감%" and Content not like "%화병%"')
freeboard = cur.fetchall()

con.commit()
con.close()

In [2]:
print("조현병-자유게시판:" + str(len(schizophrenia_free)))
print("조현병-상담실:" + str(len(schizophrenia_qna)))
print("ADHD-자유게시판:" + str(len(adhd_free)))
print("ADHD-익명게시판:" + str(len(adhd_anonymous)))
print("우울증-자유게시판:" + str(len(sad)))

조현병-자유게시판:2360
조현병-상담실:11364
ADHD-자유게시판:1390
ADHD-익명게시판:3727
우울증-자유게시판:9855


In [8]:
print("일반 자유게시판:" + str(len(freeboard)))

일반 자유게시판:7845


In [3]:
schizophrenia = schizophrenia_free + schizophrenia_qna
adhd = adhd_free + adhd_anonymous

## Sen2Vec

In [6]:
import sen2vec
sen2vec = sen2vec.Sentence2Vec()
sad_list = []
sad_list = sen2vec.get_vector(sad)

In [9]:
import sen2vec
sen2vec = sen2vec.Sentence2Vec()
adhd_list = []
adhd_list = sen2vec.get_vector(adhd)

In [10]:
import sen2vec
sen2vec = sen2vec.Sentence2Vec()
schizophrenia_list = []
schizophrenia_list = sen2vec.get_vector(schizophrenia)

In [11]:
import sen2vec
sen2vec = sen2vec.Sentence2Vec()
freeboard_list = []
freeboard_list = sen2vec.get_vector(freeboard)

### 정신분열증

In [12]:
x_train = []
for i in range(0,len(schizophrenia_list),1):
    x_train.append(schizophrenia_list[i])
for i in range(0,len(freeboard_list),1):
    x_train.append(freeboard_list[i])

In [13]:
y_train = []
for i in range(0,len(schizophrenia_list),1):
    y_train.append(0)
for i in range(0,len(freeboard_list),1):
    y_train.append(1)

### ADHD

In [19]:
x_train = []
for i in range(0,len(adhd_list),1):
    x_train.append(adhd_list[i])
for i in range(0,len(freeboard_list),1):
    x_train.append(freeboard_list[i])

In [20]:
y_train = []
for i in range(0,len(adhd_list),1):
    y_train.append(0)
for i in range(0,len(freeboard_list),1):
    y_train.append(1)

### 우울증

In [26]:
x_train = []
for i in range(0,len(sad_list),1):
    x_train.append(sad_list[i])
for i in range(0,len(freeboard_list),1):
    x_train.append(freeboard_list[i])

In [27]:
y_train = []
for i in range(0,len(sad_list),1):
    y_train.append(0)
for i in range(0,len(freeboard_list),1):
    y_train.append(1)

여기까지 vector

### SVM

- 정신분열증

In [14]:
from sklearn.svm import SVC
ml = SVC(kernel='linear', C=1.0, random_state=0)
ml.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
y_pred = ml.predict(x_train)

In [16]:
from sklearn.metrics import accuracy_score
print('총 테스트 개수: %d, 오류개수: %d' %(len(y_train), (y_train != y_pred).sum()))
print('정확도: %.2f' %accuracy_score(y_train, y_pred))

총 테스트 개수: 21569, 오류개수: 1961
정확도: 0.91


In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('총 테스트 개수: %d, 오류개수: %d' %(len(y_train), (y_train != y_pred).sum()))
print('정확도: %.2f' %accuracy_score(y_train, y_pred))
print('f1_score : %.2f' %f1_score(y_train, y_pred, average='macro'))

총 테스트 개수: 21569, 오류개수: 1961
정확도: 0.91
f1_score : 0.91


In [17]:
for i in range(0,len(y_pred),1):
    print(y_pred[i])

1
1
1
1
0
0
1
1
1
1
0
1
1
0
1
0
0
1
1
0
0
0
1
0
0
1
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
1
0
1
1
1
1
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
1
1
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
1
1
0
1
0
0
0
0
0
0
1
0
1
0
0
1
0
0
1
1
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
0
0
1
0
0
1
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
1
0
0
0
0
0
1
0
1
0
0
1
0
0
0
1
0
1
1
1
1
0
1
1
0
1
0
1
1
1
0
0
0
0
0
1
0
0
1
0
1
0
0
0
0
0
1
0
1
1
1
0
1
1
1
1
0
1
0
1
1
0
0
1
1
0
1
0
1
0
1
0
0
1
0
0
0
1
0
1
0
0
0
0
1
0
0
1
1
1
0
1
0
0
0
0
0
0
1
0
1
0
1
0
0
1
0
1
0
0
0
1
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
1
0
0
1
1
0
1
1
1
0
0
1
0
1
0
0
0
0
1
0
1
0
0
1
0
0
1
0
1
1
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
1
1
0
1
0
0
0
0
0
1
1
1
1
1
0
1
0
0
0
1
1
1
1
1
0
1
1
0
0
0
0
1
1
1
1
1
0
1
1
0
1
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
0
1
1
1
0
0
0
1
0
1
1
0
0
0
0
0
1
1
0
1
0
0
0
1
0
0
1
0
0
1
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
1
1
0
0
1
1
1
1
0
0
1
1
0
0
0
0
0
1
0
1
0
1
0
0
1
1
1
0
1
0
1


0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


-  ADHD

In [21]:
from sklearn.svm import SVC
ml = SVC(kernel='linear', C=1.0, random_state=0)
ml.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
y_pred = ml.predict(x_train)

In [23]:
from sklearn.metrics import accuracy_score
print('총 테스트 개수: %d, 오류개수: %d' %(len(y_train), (y_train != y_pred).sum()))
print('정확도: %.2f' %accuracy_score(y_train, y_pred))

총 테스트 개수: 12962, 오류개수: 3386
정확도: 0.74


In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('총 테스트 개수: %d, 오류개수: %d' %(len(y_train), (y_train != y_pred).sum()))
print('정확도: %.2f' %accuracy_score(y_train, y_pred))
print('f1_score : %.2f' %f1_score(y_train, y_pred, average='macro'))

총 테스트 개수: 12962, 오류개수: 3386
정확도: 0.74
f1_score : 0.67


In [25]:
for i in range(0,len(y_pred),1):
    print(y_pred[i])

0
1
0
0
0
0
0
0
1
1
1
0
1
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
1
0
1
1
1
1
1
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
1
1
1
1
1
0
0
0
1
1
1
0
0
1
0
0
1
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
1
1
1
0
1
1
0
0
1
0
0
0
0
1
0
1
1
0
0
0
1
0
0
1
0
0
1
1
0
0
0
1
0
0
1
1
0
0
0
0
0
1
0
1
1
1
0
1
1
0
1
1
1
1
0
1
0
0
0
1
1
1
0
0
0
0
0
0
1
1
1
1
0
1
0
0
1
1
0
1
1
1
0
1
0
1
1
0
1
0
0
0
1
0
1
1
0
0
0
0
0
1
0
0
1
0
0
0
1
1
0
1
0
0
1
0
1
1
0
1
0
1
0
0
0
0
0
1
0
0
0
0
1
0
1
0
1
1
1
0
1
1
0
0
0
1
0
1
1
1
0
0
1
0
0
0
0
1
0
1
1
1
0
0
1
0
0
1
0
1
1
0
0
0
1
1
1
1
0
1
0
0
1
0
1
1
0
0
0
1
0
0
1
1
0
1
0
0
1
0
0
0
0
1
0
1
0
0
1
1
1
0
0
0
1
0
0
1
0
1
0
0
0
1
1
0
1
0
0
0
0
1
0
1
1
1
0
0
0
1
0
0
1
1
1
0
1
0
1
1
1
1
1
0
0
0
1
0
0
0
1
1
0
1
0
1
0
0
1
0
0
0
1
0
1
0
1
1
1
1
0
1
0
0
0
0
1
0
0
1
0
1
0
0
1
1
1
1
1
1
1
1
1
0
1
1
1
0
0
0
1
0
0
0
1
1
0
0
0
1
1
1
0
1
1
1
1
1
0
1
0
0
1
1
0
0
1
1
0
1
0
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
0
1
1
1
0
1
0
0
0
0
0
0
1
1
0
0
0
1
0
0
1
0
0
1
1
0
0
1
1
1
1
1
0
1


1
1
0
1
1
1
1
0
0
1
1
1
0
0
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
1
0
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
0
0
0
1
1
0
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
0
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
0
1
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
0
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
0
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
1
1
0
1
0
1
1
1
0
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


- 우울증

In [28]:
from sklearn.svm import SVC
ml = SVC(kernel='linear', C=1.0, random_state=0)
ml.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
y_pred = ml.predict(x_train)

In [30]:
from sklearn.metrics import accuracy_score
print('총 테스트 개수: %d, 오류개수: %d' %(len(y_train), (y_train != y_pred).sum()))
print('정확도: %.2f' %accuracy_score(y_train, y_pred))

총 테스트 개수: 17700, 오류개수: 5107
정확도: 0.71


In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print('총 테스트 개수: %d, 오류개수: %d' %(len(y_train), (y_train != y_pred).sum()))
print('정확도: %.2f' %accuracy_score(y_train, y_pred))
print('f1_score : %.2f' %f1_score(y_train, y_pred, average='macro'))

총 테스트 개수: 17700, 오류개수: 5107
정확도: 0.71
f1_score : 0.66


In [32]:
for i in range(0,len(y_pred),1):
    print(y_pred[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
1
1
1
1
0
0
1
1
1
1
0
0
0
0
1
1
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
1
1
1
1
0
1
1
1
0
1
0
0
0
1
0
1
0
0
1
0
1
1
0
1
1
0
0
1
1
1
1
0
0
1
1
0
0
1
1
0
0
0
1
1
0
0
0
1
1
1
0
0
0
0
1
0
1
1
0
0
1
1
0
0
1
1
0
0
1
1
1
0
0
1
1
1
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
1
0
1
1
1
0
1
0
1
0
1
0
1
0
0
1
0
1
1
0
0
1
0
0
0
0
1
0
0
1
0
1
0
0
1
1
0
1
1
1
0
0
1
1
0
0
1
0
0
1
1
0
1
1
1
1
0
1
0
0
1
0
1
0
0
0
1
0
1
1
0
1
0
0
0
1
1
1
0
0
1
0
0
0
0
0
0
0
1
0
0
0
1
1
0
1
1
1
0
0
0
0
0
0
0
0
1
1
0
1
1
1
1
0
1
1
0
1
1
0
0
0
0
1
1
1
1
1
1
0
1
1
0
0
0
0
0
1
0
0
0
0
0
1
1
0
0
1
0
0
1
0
1
0
0
0
0
1
1
0
1
1
0
1
1
0
1
0
0
0
0
1
1
1
0
1
0
1
0
1
0
0
1
1
1
0
0
1
0
0
1
1
0
1
0
0
1
1
1
1
0
0
0
0
1
0
0
0
0
0
1
1
1
1
0
0
1
1
0
0
0
1
0
1
1
1
1
0
0
1
1
1
0
0
1
0
0
1
1
0
0
1
0
0
1
1
0
1
0
1
1
1
1
0
1
1
1
0
0
1
0
0
0
1
0
0
1
1
1
1
1
0
0
0
1
1
0
1
0
1
0
1
0
1
0
1
0
0
1
1
1
1
0
0
0
0
1
0
0
1
1
0
1
0
1
0
0
1
1
0
0
1
0
0
0
1
1
1
1
1
0
1
1
0
1
1
0
1
0
0
1
0
1
1
0
1
1
0
0
0
0
0
1
1
1
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
